# Steps

1. Build a `RunOptions` generator with the parameters of interest, namely $dim_S, J_S, \delta_S, U_S, dim_A, J_A, \delta_A, U_A, \alpha_{xx}, \alpha_{xz}, \alpha_{zx}, \alpha_{zz}, t$. We need a decent range for $\delta_S$ for the Bayesian update side of things. We assume a uniform prior on the cartesian product.
2. Calculate the probabilities for the $J_z$ measurement for each value. This is a `dict`, as $J_z$ can have multiple outcomes.
3. Use the probabilities to build the expected likelihood function over all inputs, for a given range of $N_{trials}$. This also requires a "True" set of values for each parameter.
4. Use the likelihood to build the marginal distribution for all variables.
5. Save the std variation for each estimated variable!
6. Plot $\Delta$ against the $N_{trials}$, keeping the parameters of the plot saved

In [1]:
from dataclasses import dataclass, field
from src.angular_momentum import generate_spin_matrices
from tqdm import tqdm
import functools
import itertools
import numpy as np
import pandas as pd
import scipy
from matplotlib.pyplot import figure

figure(figsize=(16, 6))

# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

<Figure size 1600x600 with 0 Axes>

In [ ]:
@dataclass
class Settings:
    dim_s: int
    dim_a: int
    probability_error_tolerance: float
    system_jx: np.array = field(init=False)
    system_jz: np.array = field(init=False)
    ancilla_jx: np.array = field(init=False)
    ancilla_jz: np.array = field(init=False)
    initial_state: np.array = field(init=False)

    def __post_init__(self):
        self.system_jx, self.system_jz = generate_spin_matrices(dim=self.dim_s)
        self.ancilla_jx, self.ancilla_jz = generate_spin_matrices(dim=self.dim_a)
        self.initial_state = np.zeros(self.dim_s * self.dim_a)
        self.initial_state[0] = 1
        self.initial_state = np.outer(self.initial_state, self.initial_state)

    def generate_hamiltonian(
        self,
        j_s: float,
        u_s: float,
        delta_s: float,
        j_a: float,
        u_a: float,
        delta_a: float,
        alpha_xx: float,
        alpha_xz: float,
        alpha_zx: float,
        alpha_zz: float,
    ) -> np.array:
        system_hamiltonian = np.kron(
            -1 * j_s * self.system_jx + u_s * self.system_jz @ self.system_jz + delta_s * self.system_jz,
            np.divide(np.eye(self.dim_a), self.dim_a)
        )
        ancillary_hamiltonian = np.kron(
            np.divide(np.eye(self.dim_s), self.dim_s),
            -1 * j_a * self.ancilla_jx + u_a * self.ancilla_jz @ self.ancilla_jz + delta_a * self.ancilla_jz
        )
        interaction_hamiltonian = functools.reduce(
            lambda x, y: x + y,
            [
                alpha_xx * np.kron(self.system_jx, self.ancilla_jx),
                alpha_xz * np.kron(self.system_jx, self.ancilla_jz),
                alpha_zx * np.kron(self.system_jz, self.ancilla_jx),
                alpha_zz * np.kron(self.system_jz, self.ancilla_jz),
            ]
        )
        return system_hamiltonian + ancillary_hamiltonian + interaction_hamiltonian

    def trace_out_ancillary(self, state: np.array):
        return np.trace(
            np.array(state).reshape(self.dim_s, self.dim_a, self.dim_s, self.dim_a),
            axis1=1,
            axis2=3
        )

    @staticmethod
    def calculate_final_state(
        hamiltonian: np.array,
        initial_state: np.ndarray,
        t: float = 0,
    ) -> np.array:
        return scipy.linalg.expm(-1j * t * hamiltonian) @ initial_state @ scipy.linalg.expm(1j * t * hamiltonian)

    def calculate_probabilities(self, final_state: np.array) -> np.array:
        system_state = self.trace_out_ancillary(state=final_state)
        probabilities = [np.abs(x)**2 for x in np.diag(system_state)] # TODO: Ensure these actually up to 1!!!
        # assert np.abs(np.sum(probabilities) - 1) < settings.probability_error_tolerance, f"⚠ The observed probabilities {probabilities} are unphysical by {np.abs(np.sum(probabilities) - 1):.5f}%"
        return probabilities

# Initial state $\ket{0} \otimes \ket{0}$

In [ ]:
settings = Settings(
    dim_s= 2,
    dim_a= 1,
    probability_error_tolerance= .001,
)
pd.DataFrame(settings.initial_state).style.background_gradient(cmap='viridis')

# Build Generator Object

In [ ]:
df = pd.DataFrame([
    {
        "j_s": j_s,
        "u_s": u_s,
        "delta_s": delta_s,
        "j_a": j_a,
        "u_a": u_a,
        "delta_a": delta_a,
        "alpha_xx": alpha_xx,
        "alpha_xz": alpha_xz,
        "alpha_zx": alpha_zx,
        "alpha_zz": alpha_zz,
        "time": time
    }
    for j_s, u_s, delta_s, j_a, u_a, delta_a, alpha_xx, alpha_xz, alpha_zx, alpha_zz, time
    in itertools.product(
        np.linspace(.1999, .2001, 11), # j_s: float,
        np.linspace(.0999, .1001, 11), # u_s: float,
        np.linspace(1.1, 1.3, 1001), # delta_s: float,
        [.3], # j_a: float,
        [.1], # u_a: float,
        [1], # delta_a: float,
        [0], # alpha_xx: float,
        [0], # alpha_xz: float,
        [0], # alpha_zx: float,
        [0], # alpha_zz: float,
        np.linspace(4.99, 5.01, 3), # time: float
    )],
)
df.sample(10).style.background_gradient(cmap='viridis', axis=0)

# Define True System

In [ ]:
true_values = {
    "j_s": .2,
    "u_s": .1,
    "delta_s": 1.2,
    "j_a": .3,
    "u_a": .1,
    "delta_a": 1,
    "alpha_xx": 0,
    "alpha_xz": 0,
    "alpha_zx": 0,
    "alpha_zz": 0,
    "time": 5,
}

# Calculate measurement probabilities

In [ ]:
def calculate_measurement_probabilities(row: pd.Series) -> np.array:
    hamiltonian = settings.generate_hamiltonian(
        j_s = row["j_s"],
        u_s = row["u_s"],
        delta_s = row["delta_s"],
        j_a = row["j_a"],
        u_a = row["u_a"],
        delta_a = row["delta_a"],
        alpha_xx = row["alpha_xx"],
        alpha_xz = row["alpha_xz"],
        alpha_zx = row["alpha_zx"],
        alpha_zz = row["alpha_zz"]
    )
    final_state = settings.calculate_final_state(
        hamiltonian = hamiltonian,
        initial_state = settings.initial_state,
        t = row["time"],
    )
    # return final_state
    measurement_probabilities = settings.calculate_probabilities(final_state)
    return measurement_probabilities
    # return np.divide(final_probabilities, np.sum(final_probabilities))

df["probabilities"] = df.progress_apply(calculate_measurement_probabilities, axis=1) # i.e. Prob[J_z=k] for k in range(0, dim_s)
df["final_state_error"] = df.progress_apply(lambda row: np.abs(np.sum(row["probabilities"]) - 1), axis=1)

df.sample(30).style.background_gradient(cmap='viridis', axis=0)

# Calculate likelihoods

In [ ]:
true_likelihoods = calculate_measurement_probabilities(true_values)
# true_likelihoods /= np.prod(true_likelihoods)
true_likelihoods

# Define n_trials

In [ ]:
df = df.merge(
    pd.DataFrame([{"n_trials": 2**x} for x in range(16)]),
    how='cross'
)
df.astype({'n_trials': 'int32'}, copy=False)
df.shape

In [ ]:
vector = np.array([.1, .9])
n = 1000
scipy.special.loggamma(n * np.sum(vector)) - np.sum(scipy.special.loggamma(n * vector))
np.sum((n * vector - 1)*np.log(vector))

In [ ]:
def calculate_log_likelihood(true_probabilities: np.array, expected_probabilities: np.array, n_trials: float) -> float:
    true_probabilities = np.array(true_probabilities)
    expected_probabilities = np.array(expected_probabilities)
    # return np.sum([
    #     scipy.special.loggamma(n_trials * np.sum(expected_probabilities)),
    #     np.sum((n_trials * expected_probabilities - 1)*np.log(true_probabilities)),
    #     np.sum(scipy.special.loggamma(n_trials * expected_probabilities)),
    # ])
    return n_trials * np.sum(expected_probabilities*np.log(true_probabilities))

df["log_likelihood"] = df.progress_apply(lambda row: calculate_log_likelihood(
    true_probabilities=true_likelihoods,
    expected_probabilities=row["probabilities"],
    n_trials=row["n_trials"]
), axis=1)

In [ ]:
# df.sample(30).style.background_gradient(cmap='viridis', axis=0)
df.sort_values("log_likelihood", ascending=False).head(10).style.background_gradient(cmap='viridis', axis=0)

# Take likelihood marginals

In [ ]:
aggregates = pd.DataFrame()
for parameter in [par for par in true_values.keys() if len(set(df[par]))>=5]: # Only plot parameters for which we attempted at least 5 different values
    temp_df = df[[parameter, "n_trials", "log_likelihood"]]
    temp_df.sample(10)

    likelihood_df = temp_df\
    .groupby([parameter, "n_trials"])["log_likelihood"]\
    .agg(lambda row: np.sum(np.exp(row)))\
    .reset_index()\
    .pivot(columns="n_trials", index=parameter, values="log_likelihood")

    likelihood_df /= likelihood_df.sum(axis=0)
    # mean = np.divide(df["delta_s"] @ df["likelihood"], df["likelihood"].sum())
    likelihood_df.plot.line(figsize=(16,3), title=parameter)

    aggregates = pd.concat([aggregates, pd.DataFrame({
        "variable": parameter,
        "n_trials": likelihood_df.columns,
        "mean": likelihood_df.apply(lambda row: np.average(likelihood_df.index, weights=row), axis=0),
        "var": likelihood_df.apply(lambda row: np.average(row**2, weights=likelihood_df.index) - np.average(row, weights=likelihood_df.index)**2, axis=0)
    })])


In [ ]:
aggregates.pivot(index="n_trials", columns="variable", values="var")

In [ ]:
aggregates.pivot(index="n_trials", columns="variable", values="var").plot.line(figsize=(16,3),logx=True)